# **Shivam Krishna Mishra 20230802060 Lab 05**

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (14, 9)

def show(title, img, cmap=None):
    """Show image using matplotlib. Accepts BGR or single-channel arrays."""
    if len(img.shape) == 3 and img.shape[2] == 3:
        # convert BGR (opencv) to RGB (for display)
        display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        display = img
    plt.figure()
    plt.imshow(display, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

def imread_safe(path):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Could not load '{path}'. Put the image in working directory.")
    return img

# --- load image (BGR) ---
img_bgr = imread_safe('/content/sfv.jpg')   # replace file name if needed
h, w, _ = img_bgr.shape
print(f"Image shape (H,W,C): {img_bgr.shape}")

# Quick display
show("Original (RGB)", img_bgr)

# -------------------------------------------------------
# 1) Color space conversions
# -------------------------------------------------------

img_rgb  = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
img_hsv  = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
img_lab  = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
img_ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)

show("RGB", cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
show("HSV (visualized by converting back to RGB)",
     cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR))
show("LAB (L channel shown)", img_lab[:, :, 0], cmap='gray')

# -------------------------------------------------------
# 2) Gamma correction (gamma < 1 brighten, gamma > 1 darken)
# -------------------------------------------------------

def gamma_correction(bgr, gamma):
    inv = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv) * 255 for i in np.arange(256)]).astype('uint8')
    return cv2.LUT(bgr, table)

gamma_08 = gamma_correction(img_bgr, 0.8)     # brighten slightly
gamma_12 = gamma_correction(img_bgr, 1.2)     # darken slightly

show("Gamma 0.8 (brighter)", gamma_08)
show("Gamma 1.2 (darker)", gamma_12)

# -------------------------------------------------------
# 3) Histogram equalization (global on luminance)
# -------------------------------------------------------

ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
y, cr, cb = cv2.split(ycrcb)
y_eq = cv2.equalizeHist(y)
ycrcb_eq = cv2.merge((y_eq, cr, cb))
img_eq_global = cv2.cvtColor(ycrcb_eq, cv2.COLOR_YCrCb2BGR)

show("Histogram Equalized (global on Y channel)", img_eq_global)

# -------------------------------------------------------
# 4) Color segmentation (red detection demo)
# -------------------------------------------------------

lower_red1 = np.array([0, 70, 50])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 70, 50])
upper_red2 = np.array([180, 255, 255])

mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)
mask = mask1 | mask2

segmented = cv2.bitwise_and(img_bgr, img_bgr, mask=mask)
show("Segmented red regions", segmented)

# -------------------------------------------------------
# 8) Color smoothing and sharpening
# -------------------------------------------------------

# Bilateral Filter (edge-preserving smoothing)
bilat = cv2.bilateralFilter(img_bgr, d=9, sigmaColor=75, sigmaSpace=75)
show("Bilateral Filter (color-preserving smoothing)", bilat)

# Sharpening using Unsharp Mask
def unsharp_mask_color(bgr, kernel_size=(5,5), sigma=1.0, amount=1.0, threshold=0):
    blurred = cv2.GaussianBlur(bgr, kernel_size, sigma)
    sharpened = cv2.addWeighted(bgr, 1 + amount, blurred, -amount, 0)
    return sharpened

sharpened = unsharp_mask_color(img_bgr, kernel_size=(5,5), sigma=1.0, amount=0.8)
show("Sharpened (Unsharp Mask)", sharpened)

# -------------------------------------------------------
# 9) Blend two color-processed versions
# -------------------------------------------------------

# CLAHE (prep)
lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
l, a, b = cv2.split(lab)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
l_clahe = clahe.apply(l)
img_clahe = cv2.cvtColor(cv2.merge((l_clahe, a, b)), cv2.COLOR_LAB2BGR)

blend = cv2.addWeighted(img_bgr, 0.6, img_clahe, 0.4, 0)
show("Blend: original (0.6) + CLAHE (0.4)", blend)

# -------------------------------------------------------
# 10) Save some outputs (optional)
# -------------------------------------------------------
cv2.imwrite('img_gamma08.png', gamma_08[:, :, ::-1])  # convert BGR->RGB before saving
cv2.imwrite('img_clahe.png', img_clahe)
cv2.imwrite('img_quantized.png', img_clahe)  # screenshot showed quantized saved same variable

print("Saved outputs: img_gamma08.png, img_clahe.png, img_quantized.png (in working directory).")
print("Done. Techniques applied: color conversions, gamma, equalization, CLAHE, white balance, segmentation, quantization, smoothing & sharpening.")


Output hidden; open in https://colab.research.google.com to view.